![Image](../images/ironhack.png)

# First of all...remember the imports!!!

Ohhhh...Python is so easy and nice 'cause there's always someone else doing the hard stuff...

![Image](../images/import.jpg)

In [10]:
import requests
import pandas as pd
import re
import json
import math

# ...and remenber to leave everything nice and tidy...

![Image](../images/spaghetti.jpg)


In [68]:
api_token = '8c8f221b1edf050477c25c812f6cf4bad4130c3d'
username = 'gonalvarez05'
base_url = 'https://api.github.com/'
key = 'repos/'
owner = 'ta-data-mad/'
repo = 'dataptmad1120/'
search = 'search/issues?q=repo:'+OWNER+REPO+'+type:pr+state:{}'
pulls = 'pulls?page={}&per_page=100&state={}'
commits = 'pulls/{}/commits'
state = 'open'

In [69]:
field_list1 = ['number',
               'title',
               'state',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [70]:
field_list2 = ['student_name',
               'number',
               'lab_name',
               'state',
               'lab_status',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [71]:
field_sort1 = ['lab_status',
               'lab_name',
               'student_name']

In [72]:
field_name1 = ['Student Name',
               'PR Number',
               'Lab Name',
               'PR Status',
               'Lab Status',
               'PR Created at',
               'PR Updated at',
               'PR Closed at',
               'PR URL',
               'base repository',
               'base',
               'head repository',
               'compare',
               'Pushed at']

# Functions 

Here you may find all functions you need to create your Pipeline. Be sure you test them before you start to build your Pipeline.

![Image](../images/pieces.jpg)

In [73]:
# You can get only 100 results per page so it is important to know the number of pages you'll need.

def pages(base_url, search, state, username, api_token):
    pages = requests.get(base_url + search.format(state), auth=(username,api_token)).json()['total_count']
    if STATE == 'open':
        pages = math.ceil(pages/100)
        return pages
    elif STATE == 'closed':
        pages = math.ceil(pages/100)
        return pages

In [74]:
# Check the committs in order to know which labs are ready to be reviewed.

def get_commits(base_url, key, owner, repo, commits, pull, username, api_token):
    
    r_commits = requests.get(base_url + key + owner + repo + commits.format(pull),
                             auth=(username, api_token)).json()
    df_commits = pd.json_normalize(r_commits)
    list_commits = list(df_commits['commit.message'])
    commit = list(set([commit if commit == 'lab-finished' else 'lab-started' for commit in list_commits]))
    if 'lab-finished' in commit:
        return 'lab-finished'
    else:
        return 'lab-started'

In [75]:
# And finally get the 'pull requests'.

def get_pulls(base_url, key, owner, repo, pulls, search, state, username, api_token, field_list1):
    pulls_list = []
    max_pages = pages(base_url, search, state, username, api_token)
    for i in range(max_pages):
        r_pulls = requests.get(base_url + key + owner + repo + pulls.format(i+1, state),
                               auth=(username, api_token)).json()
        df_pulls = pd.json_normalize(r_pulls)
        pulls_list.append(df_pulls)
    df_pulls = pd.concat(pulls_list)
    df_pulls = df_pulls[field_list]
    return df_pulls

In [76]:
# But the students aren't careful with the naming...

def student_name(x):
    if ']' in x:
        x = x.split(']')
        x = x[1].replace('_', ' ').strip()
        len_x = len(x.split(' '))
        if len_x > 1:
            x = re.findall('\w[a-zA-Z áéíóúÁÉÍÓÚñÑ-]+', x)
            x = x[0].strip()
            return x
        else:
            x = 'No student name provided'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [77]:
def lab_name(x) :
    if ']' in x:
        x = x.split(']')
        x = x[0] + ']'
        x = x.strip()
        lower_case = re.findall('[A-ZÁÉÍÓÚñÑ]+', x)
        if x[0] == '[' and x[-1] == ']' and ' ' not in x and len(lower_case) == 0:
            return x
        else:
            x = 'Lab format name is incorrect'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [78]:
# ...or forget to push their work!!!

def time_parser(x):
    try:
        x = x.strip()
        x = re.findall('[0-9]+', x)
        x = ''.join(x)
        x = pd.to_datetime(x, format='%Y%m%d%H%M%S', errors='coerce')
        return x
    except:
        return 'Nothing pushed yet'

In [79]:
# Apply!!!!!!

def df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list):
    df_pulls['student_name'] = df_pulls['title'].apply(student_name)
    df_pulls['lab_name'] = df_pulls['title'].apply(lab_name)
    df_pulls['created_at'] = df_pulls['created_at'].apply(time_parser)
    df_pulls['updated_at'] = df_pulls['updated_at'].apply(time_parser)
    df_pulls['head.repo.pushed_at'] = df_pulls['head.repo.pushed_at'].apply(time_parser)
    df_pulls['lab_status'] = df_pulls.apply(lambda col: get_commits(base_url,
                                                                    key,
                                                                    owner,
                                                                    repo,
                                                                    commits,
                                                                    col['number'],
                                                                    username,
                                                                    api_token), axis=1)
    df_status = df_pulls[field_list]
    return df_status

In [80]:
# And there you have it!!!

def create_csv(df_status, field_sort, field_name):
    df_csv = df_status.sort_values(by=field_sort, ascending=False)
    df_csv.columns = field_name
    df_csv.to_csv('../data/labs_status.csv', index=False)
    return df_csv

In [81]:
lab_name(field_list2)

'Pull request is not properly named'

In [82]:
student_name('gonzalo')

'Pull request is not properly named'

In [83]:
time_parser(field_list1)

'Nothing pushed yet'

In [87]:
df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list2)

NameError: name 'df_pulls' is not defined

In [88]:
def get_pulls(base_url, key, owner, repo, pulls, search, state, username, api_token, field_list):
    pulls_list = []
    max_pages = pages(base_url, search, state, username, api_token)
    for i in range(max_pages):
        r_pulls = requests.get(base_url + key + owner + repo + pulls.format(i+1, state),
                               auth=(username, api_token)).json()
        df_pulls = pd.json_normalize(r_pulls)
        pulls_list.append(df_pulls)
    df_pulls = pd.concat(pulls_list)
    df_pulls = df_pulls[field_list]
    return df_pulls

In [97]:
df_pulls=get_pulls(base_url, key, owner, repo, pulls, search, state, username, api_token, field_list1)


,number,title,state,created_at,updated_at,closed_at,html_url,base.repo.full_name,base.ref,head.repo.full_name,head.ref,head.repo.pushed_at
0,362,[advanced-regex] Miguel González,open,2021-01-20T20:38:06Z,2021-01-20T20:38:06Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,G0nka/dataptmad1120,advanced-regex,2021-01-20T17:32:26Z
1,361,[advanced-regex] Emily Paz,open,2021-01-20T20:32:47Z,2021-01-20T20:32:47Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,EmilyPaz/dataptmad1120,advanced-regex,2021-01-20T18:14:37Z
2,360,[dataframe-calculations] Alejandra Matías,open,2021-01-20T11:18:03Z,2021-01-20T11:18:03Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,alejandramatiasmartin2/dataptmad1120,dataframe-calculations,2021-01-20T11:17:34Z
3,359,[advanced-regex] Marina Fernández,open,2021-01-19T18:20:15Z,2021-01-21T18:45:42Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,marinafdezban/dataptmad1120,advanced-regex,2021-01-21T18:45:41Z
4,358,[advanced-regex] Antonio Sánchez,open,2021-01-19T17:00:23Z,2021-01-19T17:00:23Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,Antoniosm88/dataptmad1120,advanced-regex,2021-01-19T16:58:41Z
5,357,[advanced-regex] Vicky Viana,open,2021-01-19T16:46:32Z,2021-01-19T16:46:32Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,VickyViana/dataptmad1120,advanced-regex,2021-01-18T21:09:06Z
6,356,[advanced-regex] Clara Carbo,open,2021-01-19T14:44:11Z,2021-01-19T14:44:11Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,claracarbo/dataptmad1120,advanced-regex,2021-01-19T14:43:23Z
7,355,[advanced-regex] Alejandra Matías,open,2021-01-19T12:25:39Z,2021-01-19T12:25:39Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,alejandramatiasmartin2/dataptmad1120,advanced-regex,2021-01-20T11:17:34Z
8,354,[advanced-regex] Manuel Aquino,open,2021-01-19T12:16:52Z,2021-01-19T13:01:13Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,manuaq/dataptmad1120,advanced-regex,2021-01-19T20:31:10Z
9,353,[advanced-regex]Gonzalo Álvarez,open,2021-01-19T09:11:22Z,2021-01-19T09:11:22Z,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,gonalvarez05/dataptmad1120,advanced_regex,2021-01-19T09:10:35Z


In [98]:
df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list2)

,student_name,number,lab_name,state,lab_status,created_at,updated_at,closed_at,html_url,base.repo.full_name,base.ref,head.repo.full_name,head.ref,head.repo.pushed_at
0,Miguel González,362,[advanced-regex],open,lab-finished,2021-01-20 20:38:06,2021-01-20 20:38:06,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,G0nka/dataptmad1120,advanced-regex,2021-01-20 17:32:26
1,Emily Paz,361,[advanced-regex],open,lab-finished,2021-01-20 20:32:47,2021-01-20 20:32:47,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,EmilyPaz/dataptmad1120,advanced-regex,2021-01-20 18:14:37
2,Alejandra Matías,360,[dataframe-calculations],open,lab-finished,2021-01-20 11:18:03,2021-01-20 11:18:03,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,alejandramatiasmartin2/dataptmad1120,dataframe-calculations,2021-01-20 11:17:34
3,Marina Fernández,359,[advanced-regex],open,lab-started,2021-01-19 18:20:15,2021-01-21 18:45:42,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,marinafdezban/dataptmad1120,advanced-regex,2021-01-21 18:45:41
4,Antonio Sánchez,358,[advanced-regex],open,lab-started,2021-01-19 17:00:23,2021-01-19 17:00:23,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,Antoniosm88/dataptmad1120,advanced-regex,2021-01-19 16:58:41
5,Vicky Viana,357,[advanced-regex],open,lab-finished,2021-01-19 16:46:32,2021-01-19 16:46:32,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,VickyViana/dataptmad1120,advanced-regex,2021-01-18 21:09:06
6,Clara Carbo,356,[advanced-regex],open,lab-started,2021-01-19 14:44:11,2021-01-19 14:44:11,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,claracarbo/dataptmad1120,advanced-regex,2021-01-19 14:43:23
7,Alejandra Matías,355,[advanced-regex],open,lab-finished,2021-01-19 12:25:39,2021-01-19 12:25:39,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,alejandramatiasmartin2/dataptmad1120,advanced-regex,2021-01-20 11:17:34
8,Manuel Aquino,354,[advanced-regex],open,lab-finished,2021-01-19 12:16:52,2021-01-19 13:01:13,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,manuaq/dataptmad1120,advanced-regex,2021-01-19 20:31:10
9,Gonzalo Álvarez,353,[advanced-regex],open,lab-finished,2021-01-19 09:11:22,2021-01-19 09:11:22,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,gonalvarez05/dataptmad1120,advanced_regex,2021-01-19 09:10:35


In [101]:
df_status= df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list2)

create_csv(df_status, field_sort1, field_name1)


,Student Name,PR Number,Lab Name,PR Status,Lab Status,PR Created at,PR Updated at,PR Closed at,PR URL,base repository,base,head repository,compare,Pushed at
46,Manuel Aquino,178,[mysql-select],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,manuaq/dataptmad1120,mysql-select,Nothing pushed yet
24,Alvaro Rodriguez,301,[lab-errhand-listcomp],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,Alvaru89/dataptmad1120,lab-errhand-listcomp,Nothing pushed yet
22,Antonio Sánchez,333,[dataframe-calculations],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,Antoniosm88/dataptmad1120,dataframe-calculations,Nothing pushed yet
21,Jaime Vázquez,340,[code-simplicity-efficiency],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,jvazquezgalvezlp/dataptmad1120,lab-code-simplicity-efficiency,Nothing pushed yet
3,Marina Fernández,359,[advanced-regex],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,marinafdezban/dataptmad1120,advanced-regex,Nothing pushed yet
18,Jaime Vázquez,344,[advanced-regex],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,jvazquezgalvezlp/dataptmad1120,lab-advanced-regex,Nothing pushed yet
6,Clara Carbo,356,[advanced-regex],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,claracarbo/dataptmad1120,advanced-regex,Nothing pushed yet
4,Antonio Sánchez,358,[advanced-regex],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,Antoniosm88/dataptmad1120,advanced-regex,Nothing pushed yet
32,Miguel González,211,[advanced-mysql],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,G0nka/dataptmad1120,advanced-mysql,Nothing pushed yet
31,Jaime Vázquez,222,[advanced-mysql],open,lab-started,Nothing pushed yet,Nothing pushed yet,None,https://github.com/ta-data-mad/dataptmad1120/p...,ta-data-mad/dataptmad1120,master,jvazquezgalvezlp/dataptmad1120,lab-advanced-mysql,Nothing pushed yet
